In [3]:
!ls ../output/refine/

1


In [32]:
!python deepunet_refine.py --mode export --checkpoint ../output/refine/1/ --output_dir ../output/refine/1/frozen --ngf 16 --crop_size 1024

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
loaded lab_colorization = False
loaded ndf = 64
loaded ngf = 16
loaded which_direction = AtoB
aspect_ratio = 1.0
batch_size = 1
beta1 = 0.5
checkpoint = ../output/refine/1/
crop_size = 1024
deepunet = False
display_freq = 0
drop_rate = 0.5
flip = False
gan_weight = 1.0
gpu_fraction = 0.8
input_dir = data/croped

In [33]:
import tensorflow as tf
import cv2
import numpy as np
import os
from glob import glob

def get_tensor_by_name(name):
    name_on_device = '{}:0'.format(name)
    return tf.get_default_graph().get_tensor_by_name(name_on_device)

tf.reset_default_graph()
checkpoint = '../output/refine/1/frozen/'
meta_path = glob(checkpoint+'*.meta')[0]
print('meta path:', meta_path)
assert os.path.exists(meta_path)

tf.train.import_meta_graph(meta_path)
sess = tf.Session()
saver = tf.train.Saver(max_to_keep=1)
# print('restore: ', tf.train.latest_checkpoint(checkpoint))
saver.restore(sess, tf.train.latest_checkpoint(checkpoint))

meta path: ../output/refine/1/frozen/export.meta
INFO:tensorflow:Restoring parameters from ../output/refine/1/frozen/export


In [34]:
inputs = get_tensor_by_name('inputs')
outputs = get_tensor_by_name('outputs')
print(outputs.shape)

(?, ?, 3)


In [35]:
paths = glob('../output/shalowunet/1/frozen/temp1/*')
print(paths)

['../output/shalowunet/1/frozen/temp1/5', '../output/shalowunet/1/frozen/temp1/IMG_9271', '../output/shalowunet/1/frozen/temp1/6', '../output/shalowunet/1/frozen/temp1/8', '../output/shalowunet/1/frozen/temp1/IMG_9274', '../output/shalowunet/1/frozen/temp1/IMG_9269', '../output/shalowunet/1/frozen/temp1/IMG_9272', '../output/shalowunet/1/frozen/temp1/1', '../output/shalowunet/1/frozen/temp1/7', '../output/shalowunet/1/frozen/temp1/IMG_9265', '../output/shalowunet/1/frozen/temp1/9', '../output/shalowunet/1/frozen/temp1/IMG_9268', '../output/shalowunet/1/frozen/temp1/2', '../output/shalowunet/1/frozen/temp1/10', '../output/shalowunet/1/frozen/temp1/3', '../output/shalowunet/1/frozen/temp1/IMG_9264', '../output/shalowunet/1/frozen/temp1/11', '../output/shalowunet/1/frozen/temp1/4', '../output/shalowunet/1/frozen/temp1/IMG_9270', '../output/shalowunet/1/frozen/temp1/IMG_9263', '../output/shalowunet/1/frozen/temp1/12']


In [36]:
def read_inp(p_in):
    def read_image(path, img_name='output1', color=cv2.COLOR_BGR2GRAY):
    #     print(path, img_name+'.png')
        path = os.path.join(path, img_name+'.png')
        assert os.path.exists(path)
        img = cv2.imread(path)
        return cv2.cvtColor(img, color)
    
    img_inp1 = read_image(p_in, 'input')
    img_inp2 = read_image(p_in, 'output1')
    img_inp3 = read_image(p_in, 'output2')
    img_inp = np.stack([img_inp1,img_inp2,img_inp3], axis=-1)
    return img_inp

In [37]:
for path in paths:
    img_inp = read_inp(path)
    rv = sess.run(outputs, {inputs:img_inp})
    cv2.imwrite(os.path.join(path, 'refine_line.png'), rv[:,:,0])
    